# Práctica Background Subtraction

En esta segunda práctica del bloque de análisis de secuencias haremos trabajo con 3 técnicas de background subtraction.

En primer lugar debéis implementar vuestro propio método de eliminación de fondo basado en la idea que explicamos del filtro exponencial. Podéis hacer las adaptaciones que encontréis necesario a la idea general del método para mejorar su rendimiento.

En segundo lugar debéis de usar las implementaciones del MOG y del MOG2 que se encuentran en la librería
 [OpenCV](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_video/py_bg_subtraction/py_bg_subtraction.html).

En tercer lugar lo que quiero es que hagáis una comparativa entre su método y los dos basados en Mixture de gausiano utilizando el dataset de la competición [changedetection.net](http://changedetection.net/) del año 2012.

En concreto debe usar una secuencia de cada categoría exceptuando la de *Thermal*.

Las métricas que a usar para cada secuencia son:

* **TP** : True Positive
* **FP** : False Positive
* **FN** : False Negative
* **TN** : True Negative
* **Re (Recall)** : TP / (TP + FN)
* **F-Measure** : (2 * Precision * Recall) / (Precision + Recall)
* **Precision** : TP / (TP + FP)

Finalmente se pide que hagáis un análisis de los resultados obtenidos por cada secuencia: problemas, carencias, puntos fuertes de los algoritmos ...

**Entrega:**

* Explicación del algoritmo implementado y código fuente.
* Resultados de las secuencias en formato imagen o video.
* Resultados de las secuencias en formato tabla resumen de las métricas.
* Análisis de los resultados

### Filtro exponencial para _background subtraction_
Parte de la idea de eliminación de fondo más básica: restar el fotograman **_fondo_** (_background_) del fotograma actual para obtener el **_primer plano_** (_foreground_).  
El primer problema o tarea para realizar este algoritmo es conseguir ese fotograma **_fondo_**. Para ello se necesita una imagen que represente el fondo sin ningún otro objeto, pero esto no siempre es posible, por lo tanto hay que buscar otras soluciones. Una de ellas es estimar el fondo como la media o la mediana a partir de los primeros _n_ fotogramas.  
El segundo problema que surge es que el fondo en los casos prácticos varía con el tiempo y por lo tanto es posible que ese fotograma _fondo_ que hemos estimado no nos sirve más tarde. Por lo tanto la mejora que se propone es estimar el fondo recursivamente respecto el fotograma anterior siguiendo la siguiente fórmula: 
$$
\begin{align}
B_t=(1-{\alpha})B_{t-1}+{\alpha}I_t
\end{align}
$$
Donde $B_t$ es la estimación del fondo para el fotograma actual, $B_{t-1}$ es el fondo para el fotograma anterior, $I_t$ es el fotograma actual y $\alpha$ es el factor de aprendizaje que indica cuánto del fotograma actual se considerará fondo para el siguiente fotograma.

In [1]:
from utils import *

Para mantener la limpieza del _notebook_ se han definidos las funciones para realizar el análisis dentro del archivo _utils.py_.  
Las funciones que se encuentran son las siguientes:
- **comparator**: compara un _frame_ resultante de un modelo de _background subtraction_ con el resultado esperado (_groundtruth_)
- **loadImages**: devuelve una lista con las imágenes dentro de la carpeta especificada
- **exponentialFilter**: implementación del algoritmo basado en filtro exponencial
- **MOG**: implementación del algoritmo MOG disponible [aquí](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_video/py_bg_subtraction/py_bg_subtraction.html)
- **MOG2**: implementación del algoritmo MOG2 disponible [aquí](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_video/py_bg_subtraction/py_bg_subtraction.html)
- **im2vid**: genera video a partir de las imágenes ubicadas en una carpeta
- **showVideo**: muestra el video en una ventana flotante para visualizar mejor el resultado del modelo

Notar que, a pesar de tratarse de algoritmos que trabajan con video, el _dataset_ que se proporciona para realizar las pruebas son imágenes que representan los fotogramas de una secuencia. Por lo tanto, las funciones que se han realizado, se han pensado para trabajar con imágenes en vez de video.

In [3]:
showVideo('resultTest.mp4')

In [21]:
from utils import *

In [2]:
loadPath = 'DATA/baseline/baseline/highway/input/*.jpg'
gtHighwayPath = 'DATA/baseline/baseline/highway/groundtruth/*.png'
savePath = 'DATA/baseline/results/highway_MOG/'
images = loadImages(loadPath)
gt_hihgway_frames = loadImages(gtHighwayPath)
#exponentialFilter(images,0.05,savePath)
MOG(images, savePath)
result_MOG = loadImages('DATA/baseline/results/highway_MOG/*jpg')

tp =  np.zeros((len(result_MOG)))
fn =  np.zeros((len(result_MOG)))
fp =  np.zeros((len(result_MOG)))
tn =  np.zeros((len(result_MOG)))

for i in range(len(result_MOG)):
    tp[i],fn[i],fp[i],tn[i] = comparator(result_MOG[i],gt_hihgway_frames[i])
    print(tp[i],fn[i],fp[i],tn[i])

0.0 0.0 76800.0 0.0
0.0 1.0 76773.0 0.0
0.0 10.0 76672.0 0.0
0.0 19.0 76567.0 0.0
0.0 61.0 76391.0 0.0
0.0 74.0 76300.0 0.0
0.0 97.0 76270.0 0.0
0.0 99.0 76335.0 0.0
0.0 98.0 76335.0 0.0
0.0 126.0 76226.0 0.0
0.0 132.0 76180.0 0.0
0.0 114.0 76161.0 0.0
0.0 151.0 76181.0 0.0
0.0 162.0 76200.0 0.0
0.0 148.0 76207.0 0.0
0.0 176.0 76134.0 0.0
0.0 182.0 76216.0 0.0
0.0 191.0 76164.0 0.0
0.0 177.0 76140.0 0.0
0.0 197.0 76131.0 0.0
0.0 202.0 76074.0 0.0
0.0 212.0 76032.0 0.0
0.0 205.0 76077.0 0.0
0.0 226.0 76029.0 0.0
0.0 236.0 76025.0 0.0
0.0 245.0 76042.0 0.0
0.0 252.0 76040.0 0.0
0.0 233.0 76009.0 0.0
0.0 240.0 75992.0 0.0
0.0 246.0 76058.0 0.0
0.0 260.0 76048.0 0.0
0.0 280.0 76020.0 0.0
0.0 254.0 76041.0 0.0
0.0 255.0 76022.0 0.0
0.0 279.0 75947.0 0.0
0.0 254.0 75843.0 0.0
0.0 268.0 75824.0 0.0
0.0 292.0 75856.0 0.0
0.0 285.0 75937.0 0.0
0.0 286.0 75896.0 0.0
0.0 315.0 75925.0 0.0
0.0 291.0 75922.0 0.0
0.0 312.0 75834.0 0.0
0.0 351.0 75821.0 0.0
0.0 354.0 75818.0 0.0
0.0 360.0 75831.0 0.0

0.0 1198.0 72922.0 0.0
0.0 1165.0 72945.0 0.0
0.0 1239.0 72618.0 0.0
0.0 1349.0 72619.0 0.0
0.0 1300.0 72939.0 0.0
0.0 1329.0 72853.0 0.0
0.0 1403.0 72875.0 0.0
0.0 1464.0 72830.0 0.0
0.0 1516.0 72574.0 0.0
0.0 1581.0 72658.0 0.0
0.0 1517.0 72732.0 0.0
0.0 1599.0 72861.0 0.0
0.0 1701.0 72720.0 0.0
0.0 1838.0 72429.0 0.0
0.0 1791.0 72533.0 0.0
0.0 1882.0 72327.0 0.0
0.0 2124.0 72109.0 0.0
0.0 2177.0 71958.0 0.0
0.0 2281.0 72021.0 0.0
0.0 2407.0 71921.0 0.0
0.0 2300.0 71939.0 0.0
0.0 2450.0 71724.0 0.0
0.0 2666.0 71580.0 0.0
0.0 2756.0 70979.0 0.0
0.0 2902.0 70824.0 0.0
0.0 3106.0 70877.0 0.0
0.0 3275.0 70486.0 0.0
0.0 3502.0 70199.0 0.0
0.0 3524.0 70027.0 0.0
0.0 3760.0 69681.0 0.0
0.0 3807.0 69499.0 0.0
0.0 3798.0 69319.0 0.0
0.0 4124.0 68770.0 0.0
0.0 4011.0 68863.0 0.0
0.0 3908.0 69137.0 0.0
0.0 3562.0 69495.0 0.0
0.0 3228.0 69613.0 0.0
0.0 2874.0 70168.0 0.0
0.0 2734.0 70460.0 0.0
0.0 2485.0 70884.0 0.0
0.0 2287.0 71325.0 0.0
0.0 2411.0 71633.0 0.0
0.0 2357.0 71719.0 0.0
0.0 2136.0 

4520.0 8.0 2884.0 64934.0
4677.0 16.0 2897.0 64683.0
4806.0 9.0 2912.0 64297.0
5100.0 7.0 2950.0 63813.0
5191.0 22.0 3000.0 63694.0
5359.0 14.0 2983.0 63379.0
5577.0 15.0 3003.0 62983.0
5897.0 18.0 2974.0 62659.0
6110.0 25.0 3054.0 62235.0
6053.0 31.0 3212.0 62191.0
5753.0 56.0 3408.0 62176.0
5588.0 58.0 3658.0 61949.0
5468.0 57.0 3452.0 62056.0
5654.0 72.0 3487.0 61828.0
5688.0 79.0 3336.0 62035.0
5661.0 87.0 3323.0 61793.0
5508.0 88.0 3275.0 61948.0
5592.0 88.0 3260.0 61971.0
5584.0 89.0 3293.0 61723.0
5568.0 83.0 3262.0 61935.0
5411.0 74.0 3244.0 62128.0
5322.0 81.0 3323.0 62027.0
5392.0 75.0 3400.0 62713.0
5250.0 58.0 3409.0 62436.0
5300.0 65.0 3250.0 62331.0
5237.0 47.0 3251.0 62579.0
4976.0 33.0 3370.0 62842.0
5061.0 37.0 3460.0 62584.0
4811.0 66.0 3613.0 62459.0
4663.0 68.0 3638.0 62561.0
4418.0 28.0 3862.0 63249.0
4186.0 21.0 3768.0 63374.0
4177.0 23.0 3768.0 63506.0
3942.0 20.0 3661.0 63637.0
4006.0 23.0 3521.0 63817.0
4007.0 32.0 3519.0 63846.0
3715.0 49.0 3589.0 64130.0
3821

3497.0 61.0 2015.0 67455.0
3156.0 52.0 1990.0 67825.0
2860.0 41.0 2093.0 68033.0
2754.0 51.0 2044.0 68314.0
2574.0 56.0 2018.0 68521.0
2537.0 48.0 1933.0 69052.0
2406.0 52.0 1828.0 69217.0
2300.0 74.0 1726.0 69398.0
2036.0 63.0 1564.0 69934.0
1778.0 78.0 1658.0 70072.0
1483.0 87.0 1568.0 70597.0
1151.0 78.0 1612.0 71074.0
874.0 70.0 1598.0 71400.0
727.0 68.0 1425.0 71964.0
614.0 66.0 1239.0 72588.0
427.0 58.0 1129.0 73029.0
318.0 56.0 1013.0 73542.0
313.0 69.0 1011.0 73430.0
347.0 62.0 1058.0 73291.0
353.0 77.0 1022.0 73118.0
342.0 50.0 1022.0 73282.0
372.0 48.0 1112.0 73439.0
410.0 56.0 1062.0 73332.0
438.0 62.0 1163.0 73015.0
438.0 71.0 1231.0 72930.0
453.0 67.0 1241.0 72989.0
491.0 70.0 1136.0 72673.0
484.0 71.0 1205.0 72748.0
520.0 71.0 1181.0 72748.0
578.0 77.0 1273.0 72664.0
591.0 62.0 1287.0 72512.0
596.0 69.0 1253.0 72577.0
661.0 74.0 1274.0 72592.0
715.0 56.0 1248.0 72449.0
762.0 49.0 1313.0 72420.0
733.0 59.0 1273.0 72552.0
769.0 48.0 1367.0 72375.0
849.0 48.0 1314.0 72284.0


586.0 58.0 1561.0 71945.0
605.0 47.0 1506.0 72210.0
634.0 40.0 1601.0 72228.0
636.0 45.0 1630.0 72133.0
662.0 36.0 1682.0 72078.0
646.0 39.0 1784.0 72054.0
671.0 43.0 1827.0 72122.0
665.0 39.0 1847.0 72068.0
696.0 44.0 1868.0 71953.0
745.0 44.0 1915.0 71571.0
833.0 48.0 1812.0 71331.0
820.0 35.0 1958.0 71378.0
854.0 47.0 1916.0 71384.0
892.0 47.0 1913.0 71260.0
865.0 47.0 1970.0 71230.0
831.0 47.0 1938.0 71268.0
906.0 39.0 1975.0 71226.0
922.0 40.0 2004.0 71214.0
968.0 49.0 2186.0 70812.0
955.0 49.0 2235.0 70734.0
929.0 46.0 2276.0 70748.0
954.0 38.0 2344.0 70669.0
1039.0 45.0 2390.0 70575.0
1122.0 47.0 2341.0 70457.0
1136.0 42.0 2346.0 70187.0
1197.0 55.0 2414.0 69759.0
1241.0 66.0 2496.0 69630.0
1324.0 60.0 2654.0 69501.0
1430.0 52.0 2591.0 69510.0
1496.0 52.0 2734.0 69416.0
1538.0 58.0 2781.0 69116.0
1659.0 55.0 2889.0 68877.0
1759.0 53.0 2980.0 68566.0
1906.0 68.0 3003.0 68267.0
2098.0 59.0 3157.0 67816.0
2310.0 57.0 3155.0 67597.0
2490.0 55.0 3208.0 67225.0
2582.0 57.0 3057.0 6707

In [7]:
len(result_MOG)

1700

In [10]:
result_MOG[0].shape[0:2]

(240, 320)

In [4]:
im2vid2(result_MOG,'highway.mp4')

In [11]:
from IPython.display import Video

Video("resultTest.mp4")

In [12]:
from IPython.display import HTML

HTML("""
    <video alt="test" controls>
        <source src="resultTest.mp4" type="video/mp4">
    </video>
""")

In [17]:
from ipywidgets import Video
from IPython.display import display
import base64
video = Video.from_file('resultTest.mp4')
video

Video(value=b'\x00\x00\x00\x1cftypisom\x00\x00\x02\x00isomiso2mp41\x00\x00\x00\x08free\x00\xc1r\xb5mdat\x00\x0…